In [64]:
import pandas as pd
import glob
import os

In [ ]:
# Processing Getro Data

def processGetro():
    path = r'D:\GitHub\crypto-jobs-scraper\node-scraper\data\greto'
    all_files = glob.glob(os.path.join(path , "*.csv"))

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    deduplicated = (frame.drop_duplicates(subset='Job Link', keep='first', inplace=False)
                    .drop_duplicates(subset='Getro ObjectID', keep='first', inplace=False))

    header = ["Company Name", "Job Link", "Job Location", "Job Title", "Salary Range", "Tags", "Posted Before"]
    deduplicated.to_csv(r'D:\GitHub\crypto-jobs-scraper\data\getro\all_jobs.csv', columns = header, index=False)
    
processGetro()

In [71]:
# Processing Consider Data

def processConsider():
    path = r'D:\GitHub\crypto-jobs-scraper\node-scraper\data\consider'
    all_files = glob.glob(os.path.join(path , "*.csv"))

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    
    deduplicated = (frame.drop_duplicates(subset='Job Link', keep='first', inplace=False)
                    .drop_duplicates(subset='Consider JobID', keep='first', inplace=False))
    header = ["Company Name", "Job Link", "Job Location", "Job Title", "Salary Range", "Tags", "Posted Before"]
    deduplicated.to_csv(r'D:\GitHub\crypto-jobs-scraper\data\consider\all_jobs.csv', columns = header, index=False)
    
processConsider()